In [1]:
// prio_identical example - aligned with MATLAB implementation
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*

In [2]:
// Create network
val model = Network("model")

// Block 1: nodes
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.GPSPRIO)

In [3]:
// Block 2: closed classes with different priorities
val jobclass1 = ClosedClass(model, "Class1", 6, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 4, node1, 1)
val jobclass3 = ClosedClass(model, "Class3", 4, node1, 1)
val jobclass4 = ClosedClass(model, "Class4", 1, node1, 2)

In [4]:
// Set service times at Delay node
node1.setService(jobclass1, Erlang(3.0, 2))
node1.setService(jobclass2, Exp(1.0))
node1.setService(jobclass3, Exp(1.0))
node1.setService(jobclass4, Exp(2.0))

val w1 = 12.0; node2.setService(jobclass1, Exp(30.0), w1)
val w2 = 3.0; node2.setService(jobclass2, Exp(2.0), w2)
val w3 = 5.0; node2.setService(jobclass3, Exp(12.0), w3)
val w4 = 1.0; node2.setService(jobclass4, Exp(1.0), w4)

In [5]:
// Block 3: routing - serial routing for all classes
val P = model.initRoutingMatrix()
// Use serial routing for each class
P[jobclass1]= Network.serialRouting(node1, node2)
P[jobclass2]= Network.serialRouting(node1, node2)
P[jobclass3]= Network.serialRouting(node1, node2)
P[jobclass4]= Network.serialRouting(node1, node2)
model.link(P)

In [6]:
// Solve with different solvers
CTMC(model).avgTable.print()
JMT(model,"seed", 23000,"samples",30000,"keep",true).avgTable.print()

Station  JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
-------------------------------------------------------------------------------------------
Delay    Class1    0.02478     0.02478     0.66667     0.66667     0.03718     0.03718     
Delay    Class2    0.38027     0.38027     1.00000     1.00000     0.38027     0.38027     
Delay    Class3    1.69856     1.69856     1.00000     1.00000     1.69856     1.69856     
Delay    Class4    0.33333     0.33333     0.50000     0.50000     0.66667     0.66667     
Queue1   Class1    5.97522     0.00124     160.72527   160.72527   0.03718     0.03718     
Queue1   Class2    3.61973     0.19013     9.51898     9.51898     0.38027     0.38027     
Queue1   Class3    2.30144     0.14155     1.35494     1.35494     1.69856     1.69856     
Queue1   Class4    0.66667     0.66667     1.00000     1.00000     0.66667     0.66667     
--------------------------------------------------------------------------------

In [7]:
// Save generator matrix to .mat file
val generator = CTMC(model).getGenerator()

In [8]:
// Save to MAT file
import jline.api.mc.ctmc_solve
import jline.util.MatFileUtils
MatFileUtils.saveMatrix(generator.infGen, "Q", "prio_identical.mat")
MatFileUtils.saveMatrix(ctmc_solve(generator.infGen), "pi", "prio_identical.mat")